## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling.

Для начала, установим Yandex Cloud ML SDK.

In [9]:
%pip install --upgrade yandex-cloud-ml-sdk

I0000 00:00:1742470000.768423    3331 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: yandex-cloud-ml-sdk
    Found existing installation: yandex-cloud-ml-sdk 0.4.1
    Uninstalling yandex-cloud-ml-sdk-0.4.1:
      Successfully uninstalled yandex-cloud-ml-sdk-0.4.1

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere.

Создадим модель последней версии YandexGPT 5 и убедимся, что она кое-что знает про вина:

In [1]:
import os 
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ['folder_id']
api_key = os.environ['api_key']


sdk = YCloudML(folder_id=folder_id,auth=api_key)
model = sdk.models.completions("yandexgpt",model_version='rc')

In [3]:
model.run("Какое вино можно пить с устрицами?").text

'Выбор вина к устрицам зависит от ваших личных предпочтений и вида устриц. Однако есть несколько популярных сочетаний, которые могут вам понравиться:\n\n1. **Шардоне** (Chardonnay) — особенно хорошо подойдёт вино из региона Шабли (Chablis), так как оно обладает свежестью и минеральностью, которые подчёркивают вкус устриц.\n\n2. **Совиньон Блан** (Sauvignon Blanc) — особенно из регионов с прохладным климатом, таких как Марльборо (Новая Зеландия) или Сансер (Франция), может быть отличным выбором благодаря своей свежести и кислотности.\n\n3. **Альбариньо** (Albarino) — белое вино из Испании, которое отличается свежестью, фруктовостью и высокой кислотностью, что делает его хорошим дополнением к устрицам.\n\n4. **Мюскаде** (Muscadet) — белое вино из региона Луары во Франции, известное своей свежестью и минеральностью. Оно часто рекомендуется к устрицам из-за своей способности подчёркивать их вкус.\n\n5. **Просекко** (Prosecco) — итальянское игристое вино, которое может добавить лёгкости и с

## Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [53]:
def create_thread():
    return sdk.threads.create(name="Test", ttl_days=1, expiration_policy="static")

def create_assistant(model,tools=None):
    return sdk.assistants.create(model, ttl_days=1, expiration_policy="since_last_active",tools=tools)

In [4]:
thread = create_thread()
assistant =  create_assistant(model)

assistant.update(instruction="Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.")

thread.write("Привет! Какое вино посоветуете?")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Здравствуйте! Чтобы подобрать наиболее подходящее вино, мне нужно узнать немного больше о ваших предпочтениях. Вы предпочитаете белое, красное или розовое вино? Есть ли у вас любимые сорта винограда или стили вин? Также важно учитывать, с какими блюдами вы планируете сочетать вино.


In [5]:
thread.write("Я буду есть стейк!")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Отлично! Для стейка я бы порекомендовал красное вино с насыщенным вкусом и хорошей структурой. Например, можно рассмотреть вина из сортов каберне совиньон, мерло или шираз. Они отлично подчеркнут вкус мяса и добавят гармонию в сочетании с его текстурой. Если у вас есть предпочтения по стилю или региону, я могу дать более конкретные рекомендации.


In [6]:
thread.delete()
assistant.delete()

## Добавляем RAG

Для RAG будем использовать текстовую базу знаний по винам и винным регионам, которая хранится в виде множества файлов в директориях `source/wines` и `source/regions`. Пройдёмся по этим файлам и посмотрим на их длину в токенах.

In [48]:
from glob import glob
from tqdm.auto import tqdm
import pandas as pd

def get_token_count(filename):
    with open(filename,'r',encoding='utf8') as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename,encoding='utf-8') as f:
        l = len(f.read())
    return l
    
d = [
        {
            "File" : fn,
            "Tokens" : get_token_count(fn),
            "Chars" : get_file_len(fn),
            "Category" : fn.split('/')[1]
        } 
    for fn in glob('source/*/*.md')
    if fn.count('/') == 2
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars,Category
0,source/regions/Абруццо.md,499,2022,regions
1,source/regions/Азорские острова.md,409,1809,regions
2,source/regions/Аконкагуа.md,278,1182,regions
3,source/regions/Алентежу.md,320,1216,regions
4,source/regions/Апулия.md,489,1918,regions
...,...,...,...,...
125,source/wines/Совиньон блан.md,634,2578,wines
126,source/wines/Темпранильо.md,637,2323,wines
127,source/wines/Цвайгельт.md,663,2620,wines
128,source/wines/Шардоне.md,567,2407,wines


Посмотрим на среднюю, мин и макс длину фрагментов:

In [8]:
df.groupby('Category').agg({ 'Tokens' : ('min','mean','max') })

Tokens                 
            min        mean  max
Category                        
regions     153  423.340000  683
wines       264  551.366667  664

Мы видим, что фрагменты не превышают 700 токенов, и это значит, что нам не придётся прибегать к какой-либо стратегии чанкования. Это идеальная ситуация, когда текстовая база знаний разбита вручную на небольшие фрагменты текста.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [50]:
def upload_file(filename):
    return sdk.files.upload(filename,ttl_days=5,expiration_policy="static")

df["Uploaded"] = df['File'].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и вектороного поиска.

Также в процессе создания индекса мы указываем **стратегию чанкования** текстов, но в нашем случае она не будет применяться.

> Поскольку есть ограничение на 100 добавляемых в индекс файлов, то будем добавлять фрагменты по винам и по регионам по-очереди.

In [51]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy)

op = sdk.search_indexes.create_deferred(
    df[df["Category"]=="wines"]["Uploaded"],
    index_type = HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens = 1000,
            chunk_overlap_tokens = 100),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy()))
index = op.wait()

In [52]:
op = index.add_files_deferred(df[df["Category"]=="regions"]["Uploaded"])
xfiles = op.wait()

## Собираем RAG-ассистента

Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [54]:
search_tool = sdk.tools.search_index(index)

assistant = create_assistant(model, tools=[search_tool])
thread = create_thread()

instruction="""
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию
у пользователя.
"""

_ = assistant.update(instruction=instruction)

Assistant(id='fvtmvb0aut85av2dkoqa', expiration_config=ExpirationConfig(ttl_days=1, expiration_policy=<ExpirationPolicy.SINCE_LAST_ACTIVE: 2>), model=GPTModel(uri=gpt://b1gbicod0scglhd49qs0/yandexgpt/rc, config=GPTModelConfig(temperature=None, max_tokens=None, reasoning_mode=None, response_format=None)), instruction='\nТы - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина\nи рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию\nи выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию\nу пользователя.\n', max_prompt_tokens=None, tools=(SearchIndexTool(search_index_ids=('fvtorsgmaqdsepnb17g8',), max_num_results=None),), name=None, description=None, created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 3, 23, 22, 2, 38, 943138), updated_by='ajej20rll4tifkelclga', updated_at=datetime.datetime(2025, 3, 23, 22, 2, 39, 395477), expires_at=datetime.datetime(2025, 3,

In [55]:
thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)

К стейку подойдут следующие вина:

* **Пинотаж из Франшхука (ЮАР)** — универсальное вино, которое хорошо сочетается с различными блюдами, включая стейки из говядины.
* **Шираз из Франшхука (ЮАР)** — красное вино с насыщенным вкусом, которое хорошо гармонирует с перчёными стейками из говядины и баранины.
* **Черасуоло ди Виттория (Сицилия, Италия)** — гармонирует с тушёной говядиной и мясом на вертеле.
* **Красное сухое вино из долины Маллеко (Чили, регион Сур)** — хорошо сочетается со стейком на гриле.
* **Пино нуар из Орегона (США)** — будет хорошо сочетаться со стейком из лосося и красным мясом, например, говядиной по-бургундски.
* **Сира или шираз** — благодаря сильному характеру и содержанию танинов особенно хорошо дополняют блюда из красного мяса, например, ростбиф или мясные рулеты.


Посмотрим, из каких источников был получен этот ответ:

In [59]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != 'filechunk':
                continue
            print('------------------------')
            print(source.parts[0])
            
print_citations(result)

------------------------
## Франшхук 

Франшхук
Франшхук — крупная винодельческая провинция Южной Африки. Территориально относится к региону Стелленбош, расположена в 75 км от Кейптауна — столицы ЮАР. Деревню основали беглые французы в 1688 году. Получив земельные наделы, они разбили виноградники и наладили производство напитков. Сегодня Франшхук называют винной столицей ЮАР. На карте провинции можно насчитать 11 крупных винодельческих хозяйств. Деревня Франшхук известна необычным терруаром: она расположена в продолговатой долине и с трех сторон окружена высокими горами, которые защищают виноградники от ветра, излишней влаги зимой и палящего солнца летом. При этом четвертая сторона долины открыта для ветров Атлантики. Климат здесь умеренный, а температура воздуха ниже, чем в соседних областях. Виноград созревает медленнее, чем на открытой африканской местности, что делает вино более свежим и кислотным. В Франшхуке наиболее распространены известные европейские сорта, завезенные француза

In [60]:
thread.delete()

Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `source/food_wine_table.md` в формате markdown.

In [61]:
with open('source/food_wine_table.md',encoding='utf-8') as f:
    food_wine = f.readlines()
fw = ''.join(food_wine)

tokens = len(model.tokenize(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 12630, 3.3103721298495645 chars/token


Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [42]:
header=food_wine[:2]
header

['Продукт | Вино, которое подходит к этому продукту\n', '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [62]:
chunk_size = 600 * 3 # approx 600 tokens * 2 char/token

s = header.copy()
uploaded_foodwine = []
for x in food_wine[2:]:
    s.append(x)
    if len(''.join(s))>chunk_size:
        id = sdk.files.upload_bytes(''.join(s).encode(),ttl_days=5,expiration_policy="static")
        uploaded_foodwine.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_foodwine)} table chunks")

Uploaded 21 table chunks


Теперь добавим эти фрагменты в индекс:

In [63]:
op = index.add_files_deferred(uploaded_foodwine)
xfiles = op.wait()

Посмотрим, стал ли ответ системы лучше:

In [64]:
thread = create_thread()

thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)
print_citations(result)

Выбор вина к стейку зависит от его вида и степени прожарки.

1. **Стейк из мраморной нежной говядины (Филе-миньон)**: подойдут лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также можно выбрать «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

2. **Стейк из мраморной жирноватой говядины (Рибай и пр.)**:
* Для прожарки Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки.
* Для прожарки Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше.
-------------------

In [65]:
thread.delete()
assistant.delete()


In [66]:
index.delete()
for f in df['Uploaded']:
    f.delete()
for f in uploaded_foodwine:
    f.delete()